# Tape Analysis

In [12]:
import pandas as pd
import datetime


In [53]:
df = pd.read_excel('input/cvx.xlsx', index_col = "Date") #import data from excel file

In [54]:
df.drop(columns=['Bid','Ask','Sizes'],inplace=True) # delete columns


In [55]:
df.dropna(how='any',inplace=True)

In [58]:
df['Time']=pd.to_datetime(df["Time"],format='%H:%M:%S')

In [63]:
#df.groupby('Price').mean()


In [52]:
#df.sort_values(by =["Date"]) 


In [33]:
#f.sort_index(inplace = True)

In [32]:
with pd.ExcelWriter("output/output.xlsx") as writer:
   
    df.to_excel(writer, sheet_name="output")
    